In [266]:
import pymysql
import pickle
import xgboost as xgb
import numpy as np
import pandas as pd
import time
import warnings
from datetime import datetime
import joblib
from keras.models import load_model


In [267]:
# 경고 숨기기
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')
warnings.filterwarnings("ignore", message="X does not have valid feature names, but StandardScaler was fitted with feature names")


In [268]:
# 현재 시간 가져오기
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
current_time


'2023-10-30 09:23:34'

In [269]:
## 길이 기준 설정
avg_len = 1

In [270]:
## 모델 경로
rul_models_path = ['./model/rul_deep/lstm_fl_total.h5', './model/rul_deep/lstm_pd_total.h5', './model/rul_deep/lstm_ph_total.h5']
scalers_path = ['./model/rul_deep/X_scaler_fl.pkl','./model/rul_deep/X_scaler_pd.pkl','./model/rul_deep/X_scaler_ph.pkl']
abnormal_models_path = ['./model/abnormal_detect/RF_model(FL).pkl','./model/abnormal_detect/RF_model(PB).pkl','./model/abnormal_detect/RF_model(PH).pkl']
multi_scalers_path = './model/abnormal_detect/StandardScaler.pkl'

In [271]:
# 모델 및 스케일러 미리 로드
MODELS = [load_model(path) for path in rul_models_path]
ABNORMAL_MODELS = [joblib.load(path) for path in abnormal_models_path]
SCALERS = [joblib.load(path) for path in scalers_path]
multi_scaler = joblib.load(multi_scalers_path)

In [272]:
def fetch_recent_logs(length=avg_len):
    """mysql db에서 최근 로그를 가져오는 함수"""
    # mysql 데이터베이스에 연결

    connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                 user='oneday',  # DB 유저명
                                 password='1234',  # 비밀번호
                                 db='j6database',  # 사용할 DB 이름
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    try:
        with connection.cursor() as cursor:
            # 가장 최근의 데이터부터 지정한 길이만큼 가져오는 SQL 쿼리
            sql = f'''SELECT time, stage, Lot, runnum, recipe, recipe_step,
       IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT,
       ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT, FLOWCOOLFLOWRATE,
       FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
       FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
       FIXTURESHUTTERPOSITION, ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER,
       ETCHAUX2SOURCETIMER, ACTUALSTEPDURATION FROM input_data_1 ORDER BY input_time DESC LIMIT {length+51}'''
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        connection.close()

    return results

In [273]:
df = pd.read_csv("./model_data_input/test.csv")
df

,time,Tool,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,ETCHBEAMCURRENT,...,ETCHGASCHANNEL1READBACK,ETCHPBNGASREADBACK,FIXTURETILTANGLE,ROTATIONSPEED,ACTUALROTATIONANGLE,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION
0,3283834,01M02,1,9253,10511673,67,1,-0.079428,-0.958814,-0.959509,...,-1.751661,-2.783162,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
1,3283838,01M02,1,9253,10511673,67,1,-0.079419,-0.958928,-0.959651,...,-1.743905,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
2,3283842,01M02,1,9253,10511673,67,1,-0.079409,-0.959156,-0.960006,...,-1.743905,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
3,3283846,01M02,1,9253,10511673,67,1,-0.079420,-0.958928,-0.959297,...,-1.747158,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
4,3283850,01M02,1,9253,10511673,67,1,-0.079430,-0.959269,-0.959438,...,-1.747158,-2.784389,2.027848,-0.013207,-0.122437,0.0,2.395370,2.871511,3.175941,4.037785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3287814,01M02,1,9253,10511673,67,39,0.003771,-0.952672,-0.959651,...,-0.051796,0.348512,0.022131,-0.013207,-0.122437,1.0,2.398551,2.875072,3.179643,0.021044
996,3287818,01M02,1,9253,10511673,67,39,0.003771,-0.952672,-0.959438,...,-0.051796,0.348512,0.022131,-0.013207,-0.122437,1.0,2.398555,2.875075,3.179647,0.021044
997,3287822,01M02,1,9253,10511673,67,39,0.003771,-0.952729,-0.959297,...,-0.053538,0.356222,0.022131,-0.013207,-0.122437,1.0,2.398560,2.875079,3.179651,0.021044
998,3287826,01M02,1,9253,10511673,67,39,0.003771,-0.952672,-0.959509,...,-0.053538,0.356222,0.022131,-0.013207,-0.122437,1.0,2.398563,2.875083,3.179654,0.021044


In [274]:
def dict_to_array(data):
    """사전 형태의 데이터를 2차원 넘파이 배열로 변환하는 함수"""
    return np.array([list(item.values()) for item in data])

In [275]:
data

[{'stage': 1,
  'Lot': 9253,
  'runnum': 10511673,
  'recipe': 67,
  'recipe_step': 1,
  'IONGAUGEPRESSURE': -0.0794283467426016,
  'ETCHBEAMVOLTAGE': -0.958814437672802,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHSUPPRESSORVOLTAGE': -0.94619431001018,
  'ETCHSUPPRESSORCURRENT': -0.9390426794102577,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'FIXTURETILTANGLE': 2.027847975076401,
  'ROTATIONSPEED': -0.0132071874464973,
  'ACTUALROTATIONANGLE': -0.1224370708389037,
  'FIXTURESHUTTERPOSITION': 0,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'ETCHAUXSOURCETIMER': 2.8715106903677774,
  'ETCHAUX2SOURCETIMER': 3.1759407144678846,
  'ACTUALSTEPDURATION': 4.03778468798202,
  'time_diff': 0},
 {'stage': 1,
  'Lot': 9253,
  'runnum': 10511673,
  'recipe': 67,
  'recipe_step': 1,
  'IONGAUGEPRESSURE': -0.0794283467426016,
  'ETCHBEAMVOLTAGE': -0.958814437672

In [276]:
sequences = [data[i::5][:10] for i in range(50)]
sequences

[[{'stage': 1,
   'Lot': 9253,
   'runnum': 10511673,
   'recipe': 67,
   'recipe_step': 1,
   'IONGAUGEPRESSURE': -0.0794283467426016,
   'ETCHBEAMVOLTAGE': -0.958814437672802,
   'ETCHBEAMCURRENT': -0.9595093781327426,
   'ETCHSUPPRESSORVOLTAGE': -0.94619431001018,
   'ETCHSUPPRESSORCURRENT': -0.9390426794102577,
   'FLOWCOOLFLOWRATE': -2.842906518897486,
   'FLOWCOOLPRESSURE': -2.3328068957350747,
   'ETCHGASCHANNEL1READBACK': -1.751661148558742,
   'ETCHPBNGASREADBACK': -2.783162253978537,
   'FIXTURETILTANGLE': 2.027847975076401,
   'ROTATIONSPEED': -0.0132071874464973,
   'ACTUALROTATIONANGLE': -0.1224370708389037,
   'FIXTURESHUTTERPOSITION': 0,
   'ETCHSOURCEUSAGE': 2.3953704185454097,
   'ETCHAUXSOURCETIMER': 2.8715106903677774,
   'ETCHAUX2SOURCETIMER': 3.1759407144678846,
   'ACTUALSTEPDURATION': 4.03778468798202,
   'time_diff': 0},
  {'stage': 1,
   'Lot': 9253,
   'runnum': 10511673,
   'recipe': 67,
   'recipe_step': 1,
   'IONGAUGEPRESSURE': -0.0794283467426016,
   'ETC

In [277]:
len(sequences)

50

In [278]:
predictions = []
for sequence in sequences:
    transformed_data = dict_to_array(sequence)
    scaled_data_list = [scaler.transform(transformed_data) for scaler in SCALERS]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (10,) + inhomogeneous part.

In [250]:
len(scaled_data_list)

3

In [251]:
scaled_data_list[0].shape

(10, 23)

In [289]:
def add_difference_to_data(data):
    """각 row에 대해 다음 행의 첫 번째 열과 두 번째 열의 차이를 계산하여 data의 마지막 열에 추가하는 함수"""
    data[0]['time_diff'] = 0
    for i in range(len(data)-1):
        diff = data[i+1]['time'] - data[i]['time']
        data[i+1]['time_diff'] = diff
        del data[i]['time']
    return data[1:] ## 0 값은 의미 없으므로 제거

def predict_with_xgb_model_optimized(data,length=avg_len):
    """수명예측 모델로 예측하는 함수"""
    data = add_difference_to_data(data)
    # 슬라이딩 윈도우를 적용하여 시퀀스 생성
    sequences = [data[i::5][:10] for i in range(length)]
    
    predictions = []
    
    for sequence in sequences:
        transformed_data = dict_to_array(sequence)
        scaled_data_list = [scaler.transform(transformed_data) for scaler in SCALERS]
        
        sequence_predictions = []
        for model, scaled_data in zip(MODELS, scaled_data_list):
            pred_scaled = model.predict(scaled_data.reshape(-1,10,23))
            sequence_predictions.append(pred_scaled[0][0])
        
        predictions.append(sequence_predictions)
    
    return predictions[0]

In [290]:
def insert_single_data(connection, single_data):
    try:
        with connection.cursor() as cursor:
            # 현재 시간 가져오기
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO input_data_1 (time, Tool, stage, Lot, runnum, recipe, recipe_step,
       IONGAUGEPRESSURE, ETCHBEAMVOLTAGE, ETCHBEAMCURRENT,
       ETCHSUPPRESSORVOLTAGE, ETCHSUPPRESSORCURRENT, FLOWCOOLFLOWRATE,
       FLOWCOOLPRESSURE, ETCHGASCHANNEL1READBACK, ETCHPBNGASREADBACK,
       FIXTURETILTANGLE, ROTATIONSPEED, ACTUALROTATIONANGLE,
       FIXTURESHUTTERPOSITION, ETCHSOURCEUSAGE, ETCHAUXSOURCETIMER,
       ETCHAUX2SOURCETIMER, ACTUALSTEPDURATION, input_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, "{current_time}")'''
            cursor.execute(sql, single_data)
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()
    return current_time

In [291]:
def insert_single_rul_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO rul_1(rul_fl, rul_pb, rul_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()

In [292]:
def insert_single_multi_data(connection, data, current_time):
    try:
        with connection.cursor() as cursor:
            # 데이터 삽입 SQL.
            sql = f'''INSERT INTO multi_1(multi_pred_fl, multi_pred_pb, multi_pred_ph, input_time) 
                      VALUES (%s, %s, %s, "{current_time}")'''
            cursor.execute(sql, (data[0], data[1], data[2]))
        connection.commit()
    except Exception as e:
        print(f"Error while inserting data: {e}")
        connection.rollback()

In [293]:

connection = pymysql.connect(host='limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com',  # DB 주소
                                user='oneday',  # DB 유저명
                                password='1234',  # 비밀번호
                                db='j6database',  # 사용할 DB 이름
                                charset='utf8mb4',
                                cursorclass=pymysql.cursors.DictCursor)

# CSV 파일 읽기
df = pd.read_csv("./model_data_input/test.csv")


In [294]:

# DataFrame에서 튜플 리스트로 데이터 변환
data_tuples = list(df.itertuples(index=False, name=None))

single_data = data_tuples[0]

In [295]:

## 데이터를 한줄 씩 밀어넣으면서 진행하는 방식

current_time = insert_single_data(connection, single_data)
current_time


'2023-10-30 09:24:50'

In [296]:
data = fetch_recent_logs(length=avg_len)
data


[{'time': 3283834,
  'stage': 1,
  'Lot': 9253,
  'runnum': 10511673,
  'recipe': 67,
  'recipe_step': 1,
  'IONGAUGEPRESSURE': -0.0794283467426016,
  'ETCHBEAMVOLTAGE': -0.958814437672802,
  'ETCHBEAMCURRENT': -0.9595093781327426,
  'ETCHSUPPRESSORVOLTAGE': -0.94619431001018,
  'ETCHSUPPRESSORCURRENT': -0.9390426794102577,
  'FLOWCOOLFLOWRATE': -2.842906518897486,
  'FLOWCOOLPRESSURE': -2.3328068957350747,
  'ETCHGASCHANNEL1READBACK': -1.751661148558742,
  'ETCHPBNGASREADBACK': -2.783162253978537,
  'FIXTURETILTANGLE': 2.027847975076401,
  'ROTATIONSPEED': -0.0132071874464973,
  'ACTUALROTATIONANGLE': -0.1224370708389037,
  'FIXTURESHUTTERPOSITION': 0,
  'ETCHSOURCEUSAGE': 2.3953704185454097,
  'ETCHAUXSOURCETIMER': 2.8715106903677774,
  'ETCHAUX2SOURCETIMER': 3.1759407144678846,
  'ACTUALSTEPDURATION': 4.03778468798202},
 {'time': 3283834,
  'stage': 1,
  'Lot': 9253,
  'runnum': 10511673,
  'recipe': 67,
  'recipe_step': 1,
  'IONGAUGEPRESSURE': -0.0794283467426016,
  'ETCHBEAMVOLTA

In [297]:
len(data),len(data[0])

(52, 23)

In [298]:
rul_predictions_raw = predict_with_xgb_model_optimized(data)
rul_predictions_raw

1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 20ms/step


[557142.4, 573449.06, 9008484.0]

In [262]:
len(rul_predictions_raw)

1

In [263]:
print(len(rul_predictions_raw))
rul_predictions_raw[0][0]

1


array([537845.4], dtype=float32)

In [236]:
def compute_moving_average(data, window_size=avg_len):
    """이동평균 예측 함수"""
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


In [238]:
rul_predictions = [compute_moving_average(pred, window_size=len(data)) for pred in np.array(rul_predictions_raw).T.reshape(3,500)]
rul_predictions

[array([540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633,
        540309.81601633, 540309.81601633, 540309.81601633, 540309.81601633]),
 array([54

In [199]:
len(rul_predictions[0])

507

In [ ]:
insert_single_rul_data(connection, rul_predictions, current_time)

